In [6]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umpa-learn
!pip install -U sentencepiece
!pip install -U urllib3
!pip install -U py7zr





  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement umpa-learn (from versions: none)
ERROR: No matching distribution found for umpa-learn
You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


^C


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.7.2 requires urllib3[socks]~=1.26, but you have urllib3 2.1.0 which is incompatible.
requests 2.28.1 requires urllib3<1.27,>=1.21.1, but you have urllib3 2.1.0 which is incompatible.
botocore 1.34.13 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.1.0 which is incompatible.
You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
from datasets import load_dataset

dataset=load_dataset("cnn_dailymail", version="3.0.0")




c:\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (5.1.0)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(
c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python310\lib\site-packages\datasets\load.py:1429: FutureWarning: The repository for cnn_dailymail contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cnn_dailymail
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [9]:
dataset["train"][0]["highlights"] #summary

"Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund ."

In [10]:
'''
BART -> FACEBOOK
'''

'\nBART -> FACEBOOK\n'

In [11]:

text=dataset["train"][0]["article"][:2000]
query=text+"\nTL;DR:\n"


In [12]:
#BaRT
from transformers import pipeline, set_seed
generator_bart = pipeline('summarization', model='facebook/bart-large-cnn')
pipe_out_bart = generator_bart(text)
print(pipe_out_bart)


[{'summary_text': 'Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe says he has no plans to fritter his cash away on fast cars, drink and parties. His earnings from the first five Potter films have been held in a trust fund.'}]


In [ ]:
#FINE TUNING
from datasets import load_dataset
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import torch

DEVICE='gpu'
MODEL='facebook/bart-large-cnn'

token=AutoTokenizer.from_pretrained(MODEL)
model_use=AutoModelForSeq2SeqLM.from_pretrained(MODEL)

In [ ]:
dataset_samsun=load_dataset("samsum")
dataset_samsun['train'][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [ ]:
def get_feature(batch):
  encodings = token(batch['dialogue'], text_target=batch['summary'],
                        max_length=1024, truncation=True)

  encodings = {'input_ids': encodings['input_ids'],
               'attention_mask': encodings['attention_mask'],
               'labels': encodings['labels']}

  return encodings

# Apply the function to the dataset
samsum_pt = dataset_samsun.map(get_feature, batched=True)

samsum_pt



DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [ ]:
column=['input_ids','labels','attention_mask']
samsum_pt.set_format(type='torch',columns=column)


In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(token, model=model_use)


In [ ]:
# from datasets import Dataset

# desired_steps = 20  # The number of steps you want per epoch
# batch_size = 4      # The batch size you're using
# smaller_dataset_size = desired_steps * batch_size

# # Make sure your subset of data is correct
# smaller_dataset = samsum_pt['train'].select(range(smaller_dataset_size))


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = 'bart_samsum_ml_model',
    num_train_epochs=1,
    warmup_steps = 500,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(model=model_use, args=training_args, tokenizer=token, data_collator=data_collator,
                  train_dataset = samsum_pt['train'], eval_dataset = samsum_pt['validation'])

In [ ]:
# trainer.train()

  4%|▍         | 10/230 [1:00:40<16:54:05, 276.57s/it]

{'loss': 2.1851, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}


  9%|▊         | 20/230 [1:30:37<10:12:06, 174.89s/it]

{'loss': 1.9562, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.09}


 13%|█▎        | 30/230 [1:58:12<9:07:00, 164.10s/it] 

...

 96%|█████████▌| 220/230 [10:38:44<26:59, 161.95s/it]

{'loss': 1.3304, 'learning_rate': 2.2000000000000003e-05, 'epoch': 0.96}


100%|██████████| 230/230 [11:03:45<00:00, 173.15s/it]

{'loss': 1.3274, 'learning_rate': 2.3000000000000003e-05, 'epoch': 1.0}
{'train_runtime': 39825.2688, 'train_samples_per_second': 0.37, 'train_steps_per_second': 0.006, 'train_loss': 1.4824100038279657, 'epoch': 1.0}


TrainOutput(global_step=230, training_loss=1.4824100038279657, metrics={'train_runtime': 39825.2688, 'train_samples_per_second': 0.37, 'train_steps_per_second': 0.006, 'train_loss': 1.4824100038279657, 'epoch': 1.0})

In [ ]:
# trainer.save_model("bart_samsum_ml_model")

In [ ]:
# custome Dialogue Prediction

from transformers import pipeline

pipe = pipeline('summarization', model='bart_samsum_ml_model')
argument = {'length_penalty': 0.8, 'num_beams': 8, "max_length": 512}

custom_dialogue="""

Once, in a tranquil valley nestled between two great mountains, there was a village known as Eldoria. Eldoria was famed for its lush meadows blanketed with a myriad of flowers, its streams of crystal-clear water, and the ever-present rainbow that arched gracefully above the skyline after the gentlest of rains. The villagers led simple lives, with each day spent in harmony with nature and with each other.

In this village lived a scribe named Aelius, a man of great intellect and imagination. He spent his days chronicling the history of Eldoria, capturing the essence of the village in his finely bound books which were filled with tales of past and present. Aelius believed in the power of stories to heal, teach, and inspire. His tales were not only of the village but also of the world beyond the mountains that cradled Eldoria.

One day, a mysterious traveler arrived, her robes adorned with symbols that sparkled under the sun's caress. Her eyes held the depth of the skies, and her voice, when she spoke, was like a melody that soothed the soul. She introduced herself as Lyra and requested an audience with the village's greatest storyteller. The villagers directed her to Aelius, who welcomed her into his home, eager to hear the stories of the lands she had traversed.

Lyra spoke of distant kingdoms, of forests where trees whispered secrets to those who would listen, of oceans that harbored life in the depths unknown to the surface dwellers. She spoke of her quest to collect the greatest stories from every corner of the world and how she had heard of Aelius and his legendary tales.

Captivated by her stories, Aelius proposed a pact. He would share with her the most treasured tale of Eldoria, one that had been passed down through generations, a story of love, courage, and a dragon that breathed not fire, but stars into the night sky. In exchange, he asked to accompany her on her travels, to see with his own eyes the wonders she spoke of and to bring back new stories to his beloved village.

Lyra agreed, and on the following dawn, they set out on a journey that would become a legend in itself. They crossed the great mountains, descended into vibrant forests, and sailed across vast oceans. With every step, Aelius's quill danced on parchment, capturing the essence of their adventures.

Years passed, and the pair grew close, their bond forged through the shared love of stories and the trials they faced together. They encountered creatures of majesty and magic, survived perils of the unknown, and discovered cities that soared into the clouds.

As Aelius's collection grew, so did his fame. The stories of Eldoria's scribe spread across the lands, enchanting all who heard them. But to Aelius, the stories were more than fame; they were fragments of the world's soul, and he was merely their keeper.

Finally, as the twilight years of their lives approached, Aelius and Lyra returned to Eldoria. The village had changed little, but the eyes with which Aelius viewed it had seen the wonders of the world. He saw Eldoria as a sanctuary, a beginning, and an end to an extraordinary journey.

The stories Aelius had collected became Eldoria's greatest treasure, a beacon that drew listeners from afar. And as Aelius recounted the tales, with Lyra by his side, the villagers realized that while their valley was a small part of the world, through Aelius's stories, the world was a part of Eldoria.

And so, the story of the scribe and the traveler was woven into the tapestry of tales that Aelius had spent a lifetime crafting. Eldoria was no longer just a village in a valley but a place where stories lived and breathed, where every heart that listened could find a piece of themselves in the tales of love, adventure, and the magic that lay between the lines of Aelius's timeless stories.

"""
print(pipe(custom_dialogue, **argument))

[{'summary_text': 'Aelius is the scribe of a village called Eldoria. He spent his days chronicling the history of the village in his finely bound books filled with tales of past and present. One day, a traveler named Lyra arrived in the village. She wanted to collect the greatest stories from every corner of the world. Aelius promised to accompany her on her travels to see with his own eyes the wonders she spoke of and to bring back new stories to his beloved village.'}]
